### Creating and publishing a hugging face dataset with references to anndata files. 

Can for example be used to train multimodal models with mmcontext. The datasets consist of a reference to a sample of an anndata file,
which can be stored locally or remotly on nextcloud. 

Use the initial embedder to include some initial embeddings into our anndata object which can then be used later.

In [90]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
from adata_hf_datasets.utils import setup_logging

setup_logging()

<RootLogger root (INFO)>

In [92]:
import anndata

adata = anndata.read_h5ad("../data/demo/demo.h5ad")
adata_val = anndata.read_h5ad("../data/demo/demo_val.h5ad")

In [93]:
adata.obs["natural_language_annotation"]
caption_key = "natural_language_annotation"
batch_key = "dataset_title"  # to draw negative samples from the same batch

In [94]:
from adata_hf_datasets.embed import InitialEmbedder

methods = ["gs10k", "scvi_fm"]
# you can also input the adata_path to the InitialEmbedder, which is more memory efficient, at least for some methods
for method in methods:
    embedder = InitialEmbedder(method=method, resources_dir="../resources")
    embedder.prepare(adata)
    adata.obsm[f"X_{method}"] = embedder.embed(adata)

2025-10-16 08:18:24,688 - adata_hf_datasets.embed.initial_embedder - WARNING - GeneSelectEmbedder embedding_dim is determined by the gene list. Ignoring provided value: 64
2025-10-16 08:18:24,689 - adata_hf_datasets.embed.initial_embedder - INFO - Initialized GeneSelectEmbedder with gene_list_path: ../resources/gene_selection_10k.txt
2025-10-16 08:18:24,689 - adata_hf_datasets.embed.initial_embedder - INFO - Initialized InitialEmbedder(method=gs10k, embedding_dim=64, requires_mem_adata=True)
2025-10-16 08:18:24,692 - adata_hf_datasets.embed.initial_embedder - INFO - Preparing embedder 'gs10k'
2025-10-16 08:18:24,692 - adata_hf_datasets.embed.initial_embedder - INFO - Loading gene list from resources...
2025-10-16 08:18:24,694 - adata_hf_datasets.embed.initial_embedder - INFO - Loaded 10000 genes from ../resources/gene_selection_10k.txt
2025-10-16 08:18:24,694 - adata_hf_datasets.embed.initial_embedder - INFO - GeneSelect embedder prepared. Gene set contains 10000 genes.
2025-10-16 08:1

INFO     Loading model...                                                                                          
INFO     File /var/folders/x3/bj_4mgb56c19542qzptvc36c0000gn/T/scvi_3469d4b5d291451597aa541e8f822684/model.pt      
         already downloaded                                                                                        


/Users/mengerj/repos/adata_hf_datasets/.venv/lib/python3.12/site-packages/scvi/model/base/_base_model.py:812: UserWarning: `accelerator` has been automatically set to `cpu` although 'mps' exists. If you wish to run on mps backend, use explicitly accelerator='mps' in train function.In future releases it will become default for mps supported machines.
  _, _, device = parse_device_args(
2025-10-16 08:19:40,194 - adata_hf_datasets.embed.initial_embedder - INFO - Embedding method: scvi_fm
2025-10-16 08:19:40,195 - adata_hf_datasets.embed.initial_embedder - INFO - Preparing query AnnData and loading into SCVI model.
2025-10-16 08:19:40,200 - adata_hf_datasets.pp.pybiomart_utils - INFO - Found 17530 Ensembl IDs in index, 17530 in column 'ensembl_id'
2025-10-16 08:19:40,200 - adata_hf_datasets.pp.pybiomart_utils - INFO - Using index as source (has more/equal Ensembl IDs).
2025-10-16 08:19:40,204 - adata_hf_datasets.pp.pybiomart_utils - INFO - Final var.index has 17530 unique Ensembl IDs.


INFO     Found 41.225% reference vars in query data.                                                               


/Users/mengerj/repos/adata_hf_datasets/.venv/lib/python3.12/site-packages/scvi/model/base/_archesmixin.py:321: UserWarning: Query data contains less than 80% of reference var names. This may result in poor performance.
  return _pad_and_sort_query_anndata(adata, var_names, inplace)
/Users/mengerj/repos/adata_hf_datasets/.venv/lib/python3.12/site-packages/scvi/model/base/_archesmixin.py:108: UserWarning: `accelerator` has been automatically set to `cpu` although 'mps' exists. If you wish to run on mps backend, use explicitly accelerator='mps' in train function.In future releases it will become default for mps supported machines.
  _, _, device = parse_device_args(
/Users/mengerj/repos/adata_hf_datasets/.venv/lib/python3.12/site-packages/scvi/data/fields/_dataframe_field.py:227: UserWarning: Category 6008 in adata.obs['_scvi_batch'] has fewer than 3 cells. Models may not train properly.
  new_mapping = _make_column_categorical(
2025-10-16 08:19:40,822 - adata_hf_datasets.embed.initial_em

In [95]:
for method in methods:
    embedder = InitialEmbedder(method=method, resources_dir="../resources")
    embedder.prepare(adata_val)
    adata_val.obsm[f"X_{method}"] = embedder.embed(adata_val)

2025-10-16 08:19:41,411 - adata_hf_datasets.embed.initial_embedder - WARNING - GeneSelectEmbedder embedding_dim is determined by the gene list. Ignoring provided value: 64
2025-10-16 08:19:41,412 - adata_hf_datasets.embed.initial_embedder - INFO - Initialized GeneSelectEmbedder with gene_list_path: ../resources/gene_selection_10k.txt
2025-10-16 08:19:41,412 - adata_hf_datasets.embed.initial_embedder - INFO - Initialized InitialEmbedder(method=gs10k, embedding_dim=64, requires_mem_adata=True)
2025-10-16 08:19:41,413 - adata_hf_datasets.embed.initial_embedder - INFO - Preparing embedder 'gs10k'
2025-10-16 08:19:41,414 - adata_hf_datasets.embed.initial_embedder - INFO - Loading gene list from resources...
2025-10-16 08:19:41,415 - adata_hf_datasets.embed.initial_embedder - INFO - Loaded 10000 genes from ../resources/gene_selection_10k.txt
2025-10-16 08:19:41,416 - adata_hf_datasets.embed.initial_embedder - INFO - GeneSelect embedder prepared. Gene set contains 10000 genes.
2025-10-16 08:1

INFO     Loading model...                                                                                          
INFO     File /var/folders/x3/bj_4mgb56c19542qzptvc36c0000gn/T/scvi_fdddce55e78f4ced83dfd78581924656/model.pt      
         already downloaded                                                                                        


/Users/mengerj/repos/adata_hf_datasets/.venv/lib/python3.12/site-packages/scvi/model/base/_base_model.py:812: UserWarning: `accelerator` has been automatically set to `cpu` although 'mps' exists. If you wish to run on mps backend, use explicitly accelerator='mps' in train function.In future releases it will become default for mps supported machines.
  _, _, device = parse_device_args(
2025-10-16 08:20:13,987 - adata_hf_datasets.embed.initial_embedder - INFO - Embedding method: scvi_fm
2025-10-16 08:20:13,987 - adata_hf_datasets.embed.initial_embedder - INFO - Preparing query AnnData and loading into SCVI model.
2025-10-16 08:20:13,991 - adata_hf_datasets.pp.pybiomart_utils - INFO - Found 12022 Ensembl IDs in index, 12022 in column 'ensembl_id'
2025-10-16 08:20:13,991 - adata_hf_datasets.pp.pybiomart_utils - INFO - Using index as source (has more/equal Ensembl IDs).
2025-10-16 08:20:13,995 - adata_hf_datasets.pp.pybiomart_utils - INFO - Final var.index has 12022 unique Ensembl IDs.


INFO     Found 26.2875% reference vars in query data.                                                              


/Users/mengerj/repos/adata_hf_datasets/.venv/lib/python3.12/site-packages/scvi/model/base/_archesmixin.py:321: UserWarning: Query data contains less than 80% of reference var names. This may result in poor performance.
  return _pad_and_sort_query_anndata(adata, var_names, inplace)
/Users/mengerj/repos/adata_hf_datasets/.venv/lib/python3.12/site-packages/scvi/model/base/_archesmixin.py:108: UserWarning: `accelerator` has been automatically set to `cpu` although 'mps' exists. If you wish to run on mps backend, use explicitly accelerator='mps' in train function.In future releases it will become default for mps supported machines.
  _, _, device = parse_device_args(
/Users/mengerj/repos/adata_hf_datasets/.venv/lib/python3.12/site-packages/scvi/data/fields/_dataframe_field.py:227: UserWarning: Category 6006 in adata.obs['_scvi_batch'] has fewer than 3 cells. Models may not train properly.
  new_mapping = _make_column_categorical(
2025-10-16 08:20:14,184 - adata_hf_datasets.embed.initial_em

In [96]:
# The sample id is later used as a token for that sample. It could be seen as a cell sentence with one word.
adata.obs["sample_id_og"] = adata.obs.index
adata_val.obs["sample_id_og"] = adata_val.obs.index

In [97]:
paths = {}
adata.write_h5ad("../data/demo/demo_embed.h5ad")
paths["train"] = "../data/demo/demo_embed.h5ad"
adata_val.write_h5ad("../data/demo/demo_val_embed.h5ad")
paths["val"] = "../data/demo/demo_val_embed.h5ad"

... storing 'ensembl_id' as categorical
... storing 'ensembl_id' as categorical


In [98]:
from adata_hf_datasets.dataset import AnnDataSetConstructor
from datasets import DatasetDict

hf_dataset = DatasetDict()
# Create caption constructor with desired obs keys
for split, adata_split in zip(["train", "val"], [adata, adata_val]):
    constructor = AnnDataSetConstructor(
        dataset_format="multiplets", negatives_per_sample=2
    )
    constructor.add_anndata(
        adata_split,
        sentence_keys=[
            "sample_id_og"
        ],  # These sentences can later be tokenized to represent the sample
        caption_key=caption_key,
        batch_key=batch_key,
        adata_link=paths[split],
    )  # Can be a local path or a share link
    # Get dataset
    dataset = constructor.get_dataset()
    hf_dataset[split] = dataset

2025-10-16 08:20:15,312 - adata_hf_datasets.dataset.ds_constructor - INFO - AnnData ingested (8113 rows).
2025-10-16 08:20:15,314 - adata_hf_datasets.dataset.ds_constructor - INFO - Building dataset from generator...
2025-10-16 08:20:15,468 - adata_hf_datasets.dataset.ds_constructor - INFO - Constructed dataset with 8113 records in 'multiplets' format.
2025-10-16 08:20:15,474 - adata_hf_datasets.dataset.ds_constructor - INFO - AnnData ingested (900 rows).
2025-10-16 08:20:15,474 - adata_hf_datasets.dataset.ds_constructor - INFO - Building dataset from generator...
2025-10-16 08:20:15,494 - adata_hf_datasets.dataset.ds_constructor - INFO - Constructed dataset with 900 records in 'multiplets' format.


In [ ]:
from adata_hf_datasets.utils import annotate_and_push_dataset

embedding_generation = f"""Embeddings were generated with the InitialEmbedder class from the adata_hf_datasets package, with method = {method}, they have 
        {embedder.embedding_dim} dimensions, and are stored in adata.obsm['X_{method}']"""

dataset_type_explanation = (
    "This dataset contains multiplets of a cell sentence and a caption."
)
final_repo_id = "jo-mengr/demo_dataset"
metadata = {
    "example_share_link": paths["train"],
}
annotate_and_push_dataset(
    dataset=hf_dataset,
    embedding_generation=embedding_generation,
    dataset_type_explanation=dataset_type_explanation,
    repo_id=final_repo_id,
    readme_template_name="cellwhisperer_train",
    metadata=metadata,
    private=True,
)